# 📧 Email Campaign A/B Test Analysis
This notebook analyzes a simulated A/B test for an email campaign:

- Loads mock dataset with control (A) and variant (B)
- Calculates open, click, upgrade rates by group
- Runs 2-proportion Z-test to check significance
- Visualizes the results
- States a conclusion

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest

## Load the dataset

In [ ]:
df = pd.read_csv('email_campaign_ab_mock.csv')
df.head()

## Funnel metrics by group

In [ ]:
ab_metrics = df.groupby('experiment_group').agg(
    open_rate=('opened', 'mean'),
    click_rate=('clicked', 'mean'),
    upgrade_rate=('upgraded_to_premium', 'mean'),
    n=('user_id', 'count')
).reset_index()

ab_metrics[['open_rate', 'click_rate', 'upgrade_rate']] *= 100
ab_metrics

## Run Z-test for open rates (A vs B)

In [ ]:
opens = df.groupby('experiment_group')['opened'].sum().values
totals = df.groupby('experiment_group')['opened'].count().values

z_stat, p_val = proportions_ztest(opens, totals)
print(f"Z-statistic: {z_stat:.3f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("✅ Significant difference in open rates! (p < 0.05)")
else:
    print("❌ No significant difference.")

## Visualize group comparison

In [ ]:
melted = ab_metrics.melt(id_vars='experiment_group',
                         value_vars=['open_rate', 'click_rate', 'upgrade_rate'],
                         var_name='metric', value_name='percentage')

plt.figure(figsize=(10, 6))
sns.barplot(data=melted, x='experiment_group', y='percentage', hue='metric')
plt.title('A/B Test Funnel Metrics')
plt.ylabel('Percentage')
plt.show()

## Conclusion

Group B simulates a better subject line — the Z-test shows if the lift is significant. 
Use this result to recommend rollout or further testing!